<a href="https://colab.research.google.com/github/jproctor-rebecca/DS-Unit-1-Sprint-2-Statistics/blob/master/module2/LS_DS19_122_Chi2_Tests_Assignment_RJProctor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys 
import numpy as np
import pandas as pd
from zipfile import ZipFile
from google.colab import files
from scipy import stats
from scipy.stats import chisquare
from scipy.stats import normaltest
from scipy.stats import kruskal
from scipy.stats import ttest_ind

## Assignment - Practice Chi^2 Tests

Use the following dataset relating to math scores of students in two different Portugese schools:

<https://archive.ics.uci.edu/ml/datasets/Student+Performance>

### 1) Load the dataset specifically relating to math scores as a new dataframe.

There are two datasets in the `student.zip` file, make sure you use `student-mat.csv`.


In [2]:
# YOUR WORK HERE
# upload zipped files from url
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip


--2020-08-11 18:36:44--  https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20478 (20K) [application/x-httpd-php]
Saving to: ‘student.zip’

student.zip         100%[===================>]  20.00K  --.-KB/s    in 0.1s    

2020-08-11 18:36:45 (154 KB/s) - ‘student.zip’ saved [20478/20478]



In [3]:
# unzip files
!unzip student.zip

Archive:  student.zip
  inflating: student-mat.csv         
  inflating: student-por.csv         
  inflating: student-merge.R         
  inflating: student.txt             


In [5]:
# read data into pandas df and view data
student = pd.read_csv('student-mat.csv', sep=';')

print(student.shape)
student

(395, 33)


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,course,other,1,2,2,no,yes,yes,no,yes,yes,no,no,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,course,mother,2,1,0,no,no,no,no,no,yes,yes,no,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,course,other,1,1,3,no,no,no,no,no,yes,no,no,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,course,mother,3,1,0,no,no,no,no,no,yes,yes,no,4,4,1,3,4,5,0,11,12,10


In [6]:
# descriptive statistics
student.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.749367,2.521519,1.448101,2.035443,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,10.908861,10.713924,10.415190
std,1.276043,1.094735,1.088201,0.697505,0.839240,0.743651,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,3.319195,3.761505,4.581443
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,8.000000,9.000000,8.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,11.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000


In [9]:
student.describe(exclude='number')

,school,sex,address,famsize,Pstatus,Mjob,Fjob,reason,guardian,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic
count,395,395,395,395,395,395,395,395,395,395,395,395,395,395,395,395,395
unique,2,2,2,2,2,5,5,4,3,2,2,2,2,2,2,2,2
top,GP,F,U,GT3,T,other,other,course,mother,no,yes,no,yes,yes,yes,yes,no
freq,349,208,307,281,354,141,217,145,273,344,242,214,201,314,375,329,263


In [7]:
# exploring data for null values and dtypes
student.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   school      395 non-null    object
 1   sex         395 non-null    object
 2   age         395 non-null    int64 
 3   address     395 non-null    object
 4   famsize     395 non-null    object
 5   Pstatus     395 non-null    object
 6   Medu        395 non-null    int64 
 7   Fedu        395 non-null    int64 
 8   Mjob        395 non-null    object
 9   Fjob        395 non-null    object
 10  reason      395 non-null    object
 11  guardian    395 non-null    object
 12  traveltime  395 non-null    int64 
 13  studytime   395 non-null    int64 
 14  failures    395 non-null    int64 
 15  schoolsup   395 non-null    object
 16  famsup      395 non-null    object
 17  paid        395 non-null    object
 18  activities  395 non-null    object
 19  nursery     395 non-null    object
 20  higher    

### 2) Use Chi^2 tests and `stats.chi2_contingency()` to identify:
 - Two pairs of variables that are dependent (are associated with one another).
 - Two pairs of variables that are independent (have no significant relationship).

Does it make intuitive sense why the variables in these pairs might or might not show a relationship to one another? 


In [11]:
# YOUR WORK HERE
# 1st pair of variables that are independent 
failures_absences_contingency = pd.crosstab(student['failures'], student['absences'], margins=True)
failures_absences_contingency

absences,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,28,30,38,40,54,56,75,All
failures,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,89,3,58,6,47,4,25,6,17,3,15,2,9,2,8,0,3,1,4,0,2,1,1,1,0,0,1,0,1,0,0,1,1,1,312
1,14,0,4,1,4,0,3,1,2,0,1,0,3,1,2,2,2,0,1,1,2,0,1,0,1,1,0,1,0,1,1,0,0,0,50
2,5,0,1,0,2,0,2,0,2,0,0,1,0,0,2,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,17
3,7,0,2,1,0,1,1,0,1,0,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16
All,115,3,65,8,53,5,31,7,22,3,17,3,12,3,12,3,7,1,5,1,4,1,3,1,1,1,1,1,1,1,1,1,1,1,395


In [12]:
chi2, p_value, dof, expected = stats.chi2_contingency(failures_absences_contingency)

print("chi2 statistic", chi2)
print("p value", p_value)
print("degrees of freedom",dof)
print("expected frequencies table \n", expected)

chi2 statistic 122.18899006084001
p value 0.7958192721891358
degrees of freedom 136
expected frequencies table 
 [[9.08354430e+01 2.36962025e+00 5.13417722e+01 6.31898734e+00
  4.18632911e+01 3.94936709e+00 2.44860759e+01 5.52911392e+00
  1.73772152e+01 2.36962025e+00 1.34278481e+01 2.36962025e+00
  9.47848101e+00 2.36962025e+00 9.47848101e+00 2.36962025e+00
  5.52911392e+00 7.89873418e-01 3.94936709e+00 7.89873418e-01
  3.15949367e+00 7.89873418e-01 2.36962025e+00 7.89873418e-01
  7.89873418e-01 7.89873418e-01 7.89873418e-01 7.89873418e-01
  7.89873418e-01 7.89873418e-01 7.89873418e-01 7.89873418e-01
  7.89873418e-01 7.89873418e-01 3.12000000e+02]
 [1.45569620e+01 3.79746835e-01 8.22784810e+00 1.01265823e+00
  6.70886076e+00 6.32911392e-01 3.92405063e+00 8.86075949e-01
  2.78481013e+00 3.79746835e-01 2.15189873e+00 3.79746835e-01
  1.51898734e+00 3.79746835e-01 1.51898734e+00 3.79746835e-01
  8.86075949e-01 1.26582278e-01 6.32911392e-01 1.26582278e-01
  5.06329114e-01 1.26582278e-01 3

Chi^2 test (Independence, DoF=136)

1) Null Hypothesis: Student Failure rates have an Independent relationship with Student Absences

2) Alternative Hypothesis: Student Falure Rates have a Dependent relationship with Student Absences 

3) Confidence Level: 95%

Conclusion:

Due to a chi^2 statistic of 122.188 and a p-value of 0.795, we (reject/***fail to reject***) the null hypothesis that student failure rates have an independent relationship with student absences.

(Chi^2: FtR when p>0.05)


In [13]:
# 2nd pair of variables that are independent
failures_activities_contingency = pd.crosstab(student['failures'], student['internet'], margins=True)
failures_activities_contingency

internet,no,yes,All
failures,,,
0,47,265,312
1,13,37,50
2,2,15,17
3,4,12,16
All,66,329,395


In [14]:
chi2, p_value, dof, expected = stats.chi2_contingency(failures_internet_contingency)

print("chi2 statistic", chi2)
print("p value", p_value)
print("degrees of freedom",dof)
print("expected frequencies table \n", expected)

chi2 statistic 4.796825454131461
p value 0.7790546076737513
degrees of freedom 8
expected frequencies table 
 [[ 52.13164557 259.86835443 312.        ]
 [  8.35443038  41.64556962  50.        ]
 [  2.84050633  14.15949367  17.        ]
 [  2.67341772  13.32658228  16.        ]
 [ 66.         329.         395.        ]]


Chi^2 test (Independence, DoF=8)

1) Null Hypothesis: Student Failure rates have an Independent relationship with Student Home Internet Access

2) Alternative Hypothesis: Student Falure Rates have a Dependent relationship with Student Home Internet Access

3) Confidence Level: 95%

Conclusion:

Due to a chi^2 statistic of 4.796 and a p-value of 0.779, we (reject/***fail to reject***) the null hypothesis that student failure rates have an independent relationship with student home internet access.

(Chi^2: FtR when p>0.05)

In [15]:
# 3rd pair of variables that are independent
failures_guardian_contingency = pd.crosstab(student['failures'], student['guardian'], margins=True)
failures_guardian_contingency

guardian,father,mother,other,All
failures,,,,
0,71,230,11,312
1,14,25,11,50
2,4,6,7,17
3,1,12,3,16
All,90,273,32,395


In [16]:
chi2, p_value, dof, expected = stats.chi2_contingency(failures_guardian_contingency)

print("chi2 statistic", chi2)
print("p value", p_value)
print("degrees of freedom",dof)
print("expected frequencies table \n", expected)

chi2 statistic 54.19591524000708
p value 2.5210642499921247e-07
degrees of freedom 12
expected frequencies table 
 [[ 71.08860759 215.63544304  25.27594937 312.        ]
 [ 11.39240506  34.55696203   4.05063291  50.        ]
 [  3.87341772  11.74936709   1.37721519  17.        ]
 [  3.64556962  11.05822785   1.29620253  16.        ]
 [ 90.         273.          32.         395.        ]]


Chi^2 test (Independence, DoF=12)

1) Null Hypothesis: Student Failure rates have an Independent relationship with Student Guardianship

2) Alternative Hypothesis: Student Falure Rates have a Dependent relationship with Student Guardianship

3) Confidence Level: 95%

Conclusion:

Due to a chi^2 statistic of 54.195 and a p-value of 2.521, we (reject/***fail to reject***) the null hypothesis that student failure rates have an independent relationship with student guardianship.

(Chi^2: FtR when p>0.05)

In [17]:
# 1st pair of variables that are dependent
failures_paid_contingency = pd.crosstab(student['failures'], student['paid'], margins=True)
failures_paid_contingency

paid,no,yes,All
failures,,,
0,153,159,312
1,37,13,50
2,10,7,17
3,14,2,16
All,214,181,395


In [18]:
chi2, p_value, dof, expected = stats.chi2_contingency(failures_paid_contingency)

print("chi2 statistic", chi2)
print("p value", p_value)
print("degrees of freedom",dof)
print("expected frequencies table \n", expected)

chi2 statistic 18.537217658927684
p value 0.017540454079544205
degrees of freedom 8
expected frequencies table 
 [[169.03291139 142.96708861 312.        ]
 [ 27.08860759  22.91139241  50.        ]
 [  9.21012658   7.78987342  17.        ]
 [  8.66835443   7.33164557  16.        ]
 [214.         181.         395.        ]]


Chi^2 test (Independence, DoF=8)

1) Null Hypothesis: Student Failure rates have an Independent relationship with Student Participation in Extra Paid Classes in Course Subject

2) Alternative Hypothesis: Student Falure Rates have a Dependent relationship with Student Participation in Extra Paid Classes in Course Subject

3) Confidence Level: 95%

Conclusion:

Due to a chi^2 statistic of 18.537 and a p-value of 0.017, we (***reject***/fail to reject) the null hypothesis that student failure rates have an independent relationship with student student participation in extra paid classes in course subject; instead proposing that student failure rates have a dependent relationship with student participation in extra paid classes in course subject.

(Chi^2: R when p<0.05)

In [19]:
# 4th pair of variables that are independent
failures_schoolsup_contingency = pd.crosstab(student['failures'], student['schoolsup'], margins=True)
failures_schoolsup_contingency

schoolsup,no,yes,All
failures,,,
0,272,40,312
1,43,7,50
2,15,2,17
3,14,2,16
All,344,51,395


In [20]:
chi2, p_value, dof, expected = stats.chi2_contingency(failures_schoolsup_contingency)

print("chi2 statistic", chi2)
print("p value", p_value)
print("degrees of freedom",dof)
print("expected frequencies table \n", expected)

chi2 statistic 0.07727541233968765
p value 0.9999999099634905
degrees of freedom 8
expected frequencies table 
 [[271.7164557   40.2835443  312.        ]
 [ 43.5443038    6.4556962   50.        ]
 [ 14.80506329   2.19493671  17.        ]
 [ 13.93417722   2.06582278  16.        ]
 [344.          51.         395.        ]]


Chi^2 test (Independence, DoF=8)

1) Null Hypothesis: Student Failure rates have an Independent relationship with Student Participation in Extra Educational Support

2) Alternative Hypothesis: Student Falure Rates have a Dependent relationship with Student Participation in Extra Educational Support

3) Confidence Level: 95%

Conclusion:

Due to a chi^2 statistic of 0.077 and a p-value of 0.999, we (reject/***fail to reject***) the null hypothesis that student failure rates have an independent relationship with student student participation in extra educational support.

(Chi^2: FtR when p>0.05)

In [27]:
# 2nd pair of variables that are dependent
failures_Medu_contingency = pd.crosstab(student['failures'], student['Medu'], margins=True)
failures_Medu_contingency

Medu,0,1,2,3,4,All
failures,,,,,,
0,2,35,81,73,121,312
1,0,15,14,14,7,50
2,0,2,5,7,3,17
3,1,7,3,5,0,16
All,3,59,103,99,131,395


In [28]:
chi2, p_value, dof, expected = stats.chi2_contingency(failures_Medu_contingency)

print("chi2 statistic", chi2)
print("p value", p_value)
print("degrees of freedom",dof)
print("expected frequencies table \n", expected)

chi2 statistic 44.45276808268605
p value 0.0013083168321833623
degrees of freedom 20
expected frequencies table 
 [[2.36962025e+00 4.66025316e+01 8.13569620e+01 7.81974684e+01
  1.03473418e+02 3.12000000e+02]
 [3.79746835e-01 7.46835443e+00 1.30379747e+01 1.25316456e+01
  1.65822785e+01 5.00000000e+01]
 [1.29113924e-01 2.53924051e+00 4.43291139e+00 4.26075949e+00
  5.63797468e+00 1.70000000e+01]
 [1.21518987e-01 2.38987342e+00 4.17215190e+00 4.01012658e+00
  5.30632911e+00 1.60000000e+01]
 [3.00000000e+00 5.90000000e+01 1.03000000e+02 9.90000000e+01
  1.31000000e+02 3.95000000e+02]]


Chi^2 test (Independence, DoF=20)

1) Null Hypothesis: Student Failure rates have an Independent relationship with the Student's Mother's Level of Education

2) Alternative Hypothesis: Student Falure Rates have a Dependent relationship with the Student's Mother's Level of Education

3) Confidence Level: 95%

Conclusion:

Due to a chi^2 statistic of 44.452 and a p-value of 0.001, we (***reject***/fail to reject) the null hypothesis that student failure rates have an independent relationship with the studen's mother's level of education; instead proposing that student failure rates have a dependent relationship with the student's mother's level of education.

(Chi^2: R when p<0.05)

### 3) Use NumPy to perform your own chi^2 test "from scratch" 

Pick any of the chi^2 tests that you ran in part 2 and try them on your own. You should get the same results that Scipy got for all four values returned from `chi2_contingency()`

In [30]:
# YOUR WORK HERE
# contingency table, without margins
contingency = pd.crosstab(student['failures'], student['Medu'])
contingency

Medu,0,1,2,3,4
failures,,,,,
0,2,35,81,73,121
1,0,15,14,14,7
2,0,2,5,7,3
3,1,7,3,5,0


In [46]:
# observed values in np array
observed = contingency.values
observed

array([[  2,  35,  81,  73, 121, 312],
       [  0,  15,  14,  14,   7,  50],
       [  0,   2,   5,   7,   3,  17],
       [  1,   7,   3,   5,   0,  16],
       [  3,  59, 103,  99, 131, 395]])

In [32]:
# contingency table with margins
contingency = pd.crosstab(student['failures'], student['Medu'], margins=True)
contingency

Medu,0,1,2,3,4,All
failures,,,,,,
0,2,35,81,73,121,312
1,0,15,14,14,7,50
2,0,2,5,7,3,17
3,1,7,3,5,0,16
All,3,59,103,99,131,395


In [33]:
# calculate average number of failures of students whose mothers have no education
no_failures_no_Medu = 312*3/395
no_failures_no_Medu

2.369620253164557

In [37]:
# row totals/margins in numpy array
row_totals = student['Medu'].value_counts().sort_index().values
row_totals

array([  3,  59, 103,  99, 131])

In [39]:
# column totals/margins in numpy array
col_totals = student['failures'].value_counts().sort_index().values
col_totals

array([312,  50,  17,  16])

In [41]:
# calculate/define sample size
sample_size = student.shape[0]
sample_size

395

In [48]:
# assign expected values to numpy array
expected = np.array([[0.0,0.0],
                     [0.0,0.0],
                     [0.0,0.0],
                     [0.0,0.0],
                     [0.0,0.0]])

for i , col in enumerate(col_totals):
  for j, row in enumerate(row_totals):
    expected_value = (row*col) / sample_size
    expected[i][j] = expected_value
    # i place holder for row position, python autofills
    # j place holder for col position, python autofills

expected

IndexError: ignored

## Stretch goals:

### 1. Refactor your code so it is elegant, readable, and holds reusable code in functions.

In [ ]:
# YOUR WORK HERE
###did this as I went###



### 2. Check For Understanding - Study and write your own explanations/definitions for these topics:

- What is a sample "estimate" in statistics?

    a sample mean is an estimator for a poulation mean - a samplemean should estimate the mean of a population

- What are hypothesis test? How are they useful?

    in a hypothesis test a parameter of a population is tested; hypothesis testing determines wether or not an event occured, if a treatment had a positive effect, if variables differ from on another, or if a variable can predict the behavior of another variable

- What is a "null hypothesis?"

    a null hypothesis is a hypothesis that states there is no difference between two parameters/characteristics of a population

- What is a p-value and what does it represent?

    a p-value is the statisctical probability of obtaining the observed results of a hypothesis test; the smaller the p-value, the stroner the evidence of returning the alternative hypothesis

- What does it mean for something to be "statistically significant?"

    proving statistical significance (or lack thereof) is the purpose of statistical testing; statistical significance means that there is a relationship between two or more variables/parameter caused by something other than chance/randomness

- What is a test statistic and how does it relate to a p-value?

    a t-test is an inferential statistical tool used to determine if parameters of a population are significantly different from the poulation; the p-value is the area under the t-distribution curve/t-probability curve and to the right of the t-statistic

- What are "degrees of freedom" and how are they calculated in a 1-sample, 2-sample, and chi^2 test for independence?

    degrees of freedom is the maximum number of independent values that can vary withou violating any logic within the system/dataset



    Degrees of Freedom:

    Chi^2:  DoF = (# of categories - 1)

    T-test: DoF = Sample Size - 1

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)